<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/fine_tuning_mixed/ask_9_pubmed_bigtweet_mixed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [0]:
base_dir = 'medical-concept-normalization'

In [0]:
os.mkdir(base_dir)

In [0]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0bash: line 1: syntax error near unexpected token `newline'
bash: line 1: `<!DOCTYPE html>'
100  9115  100  9115    0     0  41621      0 --:--:-- --:--:-- --:--:-- 41621
curl: (23) Failed writing body (289 != 1212)


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [0]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [0]:
path = Path('medical-concept-normalization/data_collection')

In [0]:
# training data set

train_csv = path/'AskAPatient.fold-9.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [11]:
# validating data set

valid = pd.read_csv(path/"AskAPatient.fold-9.validation.csv",header=None)
valid.head()

,0,1
0,3,cracking sensations in my joints
1,5,fast and slow heartbeat
2,8,arthritis
3,8,arthritis
4,8,arthritis


In [12]:
# testing data set

test = pd.read_csv(path/"AskAPatient.fold-9.test.csv",header=None)
test.head()

,0,1
0,2,painful heels
1,4,Discoloration of skin [blotchy] on the neck
2,5,my heart rate jumped to 106
3,7,goiter
4,8,arthritis


In [13]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'fine_tuning_language_model_pubmed_bigtweet_mixed.csv')

In [0]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [15]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [16]:
data_clas.add_test(test,label=0)


In [0]:
data_lm.save()
data_clas.save()

In [0]:
data_lm.train_ds.vocab.itos

In [19]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [20]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [21]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,5.217255,4.636916,0.248810,16:32


In [22]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.481948,4.297652,0.283473,16:35


In [23]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.162871,4.256380,0.287630,16:34


In [24]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.106519,4.225571,0.290327,16:37


In [0]:
learn.save_encoder('bigtweet_fold0_first')

In [26]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('bigtweet_fold0_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.841536,3.315542,0.443396,01:12


In [27]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.977566,2.857894,0.512129,01:28


In [28]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.491149,2.624653,0.541779,03:21


In [29]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.275518,2.408685,0.568733,03:28
1,1.937607,2.282520,0.587601,03:35


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.847652,2.173063,0.610512,03:32
1,1.725041,2.093636,0.628032,03:37


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.669320,2.029341,0.640162,03:16
1,1.536559,1.990511,0.646900,03:29


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.442239,1.956892,0.646900,03:41
1,1.375368,1.908202,0.653639,03:27


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.343835,1.926468,0.653639,03:24
1,1.230737,1.877011,0.649596,03:23


In [34]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.208212,1.864126,0.665768,03:28
1,1.224334,1.893826,0.660377,03:45
2,1.114233,1.847898,0.661725,03:36
3,1.036401,1.829090,0.663073,03:34


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.065310,1.835601,0.673854,03:27
1,1.083399,1.831421,0.685984,03:13
2,1.045274,1.841161,0.685984,03:32
3,1.003149,1.835278,0.683288,03:40


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.983006,1.881068,0.683288,03:28
1,0.979660,1.868689,0.688679,03:24
2,0.958361,1.817146,0.680593,03:26
3,0.899893,1.815796,0.683288,03:29


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.903004,1.860208,0.677898,03:29
1,0.982516,1.856814,0.672507,03:20
2,0.954105,1.844268,0.680593,03:20
3,0.871938,1.856884,0.692722,03:37


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.883315,1.850801,0.690027,03:37
1,0.931553,1.839031,0.685984,03:27
2,0.887647,1.822531,0.690027,03:28
3,0.824929,1.809924,0.692722,03:35


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.815818,1.827493,0.690027,03:19
1,0.872977,1.859953,0.677898,03:37
2,0.835214,1.783686,0.691375,03:21
3,0.813122,1.857650,0.687332,03:39


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.860718,1.898342,0.694070,03:37
1,0.858187,1.858619,0.696765,03:40
2,0.786210,1.858552,0.691375,03:33
3,0.836472,1.862490,0.694070,03:41


In [41]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.824661,1.956879,0.694070,03:33
1,0.873263,1.908484,0.688679,03:40
2,0.822526,1.859698,0.696765,03:34
3,0.791850,1.818038,0.700809,03:29


In [42]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.775840,1.894228,0.688679,03:24
1,0.875534,1.881943,0.691375,03:17
2,0.834699,1.887130,0.695418,03:20
3,0.832083,1.879857,0.691375,03:25


In [43]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.841029,1.978335,0.688679,03:37
1,0.859978,1.943975,0.685984,03:30


In [44]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.814610,1.898423,0.681941,03:24
1,0.833100,1.881758,0.698113,03:19


In [0]:
# save the best model

learn.save_encoder('ask-0-tuning-bigtweet')

# Part three: Predict on the test dataset

In [46]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [47]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

866
866
[686, 245, 5, 455, 8, 8, 8, 8, 8, 874, 16, 16, 20, 20, 710, 375, 27, 27, 27, 861, 31, 31, 31, 260, 37, 37, 37, 37, 37, 37, 289, 289, 896, 37, 38, 211, 332, 49, 49, 49, 100, 245, 53, 53, 246, 60, 194, 63, 384, 245, 25, 343, 289, 289, 129, 741, 710, 710, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 265, 281, 710, 710, 710, 710, 2, 281, 289, 80, 189, 491, 448, 304, 88, 660, 86, 89, 710, 91, 91, 511, 431, 431, 431, 431, 94, 94, 94, 94, 245, 29, 218, 716, 746, 746, 100, 100, 100, 100, 683, 646, 108, 108, 108, 108, 111, 111, 111, 111, 117, 896, 117, 117, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 129, 129, 129, 91, 877, 268, 268, 137, 139, 140, 142, 304, 710, 710, 158, 160, 160, 321, 773, 773, 773, 773, 773, 773, 774, 172, 190, 172, 265, 172, 172, 172, 172, 164, 710, 427, 178, 181, 809, 182, 182, 289, 190, 644, 183, 183, 183, 183, 183, 710, 187, 187, 710, 188, 190, 190, 190, 182

In [48]:
correct_num = 0

for i in range(0, len(standard_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

487
0.5623556581986143
